# Database Architecture

In the lecture we learned, that there is _no free lunch_ and different databases are designed for different purposes and therefore have advantages and disadvantages. In this python notebook, we want to gather some practical experiences
using databases.

## Indices in Action

### Background Story

A fictional company *QuickMart* is a rapidly growing online store for electronics with thousands of products across various categories. With a vast customer base, QuickMart relies on fast and efficient product searches to keep up with users’ demands and to ensure low latencies. Every time a customer searches for an item, such as “Bluetooth Headphones,” the system needs to retrieve matching products from thousands of entries in real-time.

For simplicity, QuickMart’s search function uses exact product names, making it critical to match customer queries to the Product Title column in the database. However, with a growing catalog, finding an exact match by sequentially scanning every product title is increasingly time-consuming. Developers from QuickMart reached out to you, since you are an expert in optimizing database accesses.

### Overview
1. **Naive Approach**: We are starting slowly and implementing some functions which simulates the naive approach to access the products in a database. This is how Quickmart did up to now. Since we are already familiar with _pandas_, we are using pandas as the database of our choice to apply our operations.
2. **Quick Access using the simulation of an index**: Using the naive approach, we have gained a feeling about efficiency and up to how many requests this might scale. However, Quickmart grew rapidly in the past, therefore we need another strategy for the access into the database. Here we will simulate the index access to get an impression about what indices are capable of.

### 1. Naive Approach

In [17]:
import pandas as pd

url = "https://raw.githubusercontent.com/styx0r/msb_lecture_data_management_and_business_intelligence/refs/heads/main/data/pricerunner_aggregate.csv"
products = pd.read_csv(url)

def get_product(product_title, products):
    # Given a product_title and the products, please find the row,
    # where the product_title equals the column 'Product Title' in products
    return products[products['Product Title'] == product_title]


In [ ]:
%%time

# Given the get_product function, the product and a product_title,
# how long does it take to get the information of a product?
# Note: we can use the cell magic function %%time to measure the time of the execution of this cell.

# Thing about what the database is doing in order to return one product. Is it efficient?

product_title = "aeg rcb83724vw 60/40 fridge freezer white"

get_product(product_title, products)

In [19]:
# In the backend of Quickmart, the get_product function is called not just once,
# but hundreds of times per second. So next we want to measure the
# time it takes to call the process k (e.g. 2000) times.
# So first of all we need to randomly select k products and save them to a list.

k = 2000

# Please select randomly k elements from the 'Product Title' column of products, which
# which we will user later to access the products k times.
# Since products can be accessed multiple time, please sample k products randomly
# with replacement.

random_product_titles = products['Product Title'].sample(k, replace=True)


In [ ]:
%%time

# Here we want to measure the time when we call the get_products function with
# our samples k random products.

# Please iteratively use the get_product function to get all products from random_product_titles.
# Hint: You can use a for loop. Alternatively there is an apply function which you might use.

random_products = random_product_titles.apply(lambda product_title: get_product(product_title, products))

# How long does it take for k = 2000?
# How long does it take for 1000, for 2000, ... ? How does it scale regarding time?

# Hint: When asking about scaling, we want to find out how much is the increase of a unit (e.g. time)
# when the complexity of the problem increases (e.g. k).
# Imagine you plot k against the time. If it is a straight line, it's called a linear scale.
# If for example it is a logarithmic or an exponential function it is a log scale or exponential scale, respectively.

# To solve this task, we need to change the k from the cell before and run this cell and remember the times it took.

In [ ]:
import time
import matplotlib.pyplot as plt

# Additional Task: Automate the process we just did manually.
# Please run it for incremental sample sizes k and create a plot.
# The x axis should be the sample size and the y axis the time it took to get the products.

# Hint: to measure time between lines of code in python, you can use the
# time library. After importing time, time.process_time() returns a timestamp
# in seconds at this very point in time.

# Define the different sample sizes (k values) you want to test
sample_sizes = [1, 10, 100, 1000, 2000, 3000, 4000, 5000, 10000, 20000]
times = []  # List to store time taken for each sample size

# Loop through each sample size
for k in sample_sizes:
    # Take a sample of 'Product Title' column with size k
    random_product_titles = products['Product Title'].sample(k, replace=True)

    # Record the start time
    start_time = time.perf_counter()

    # Apply the get_product function on each product title in the sample
    random_products = random_product_titles.apply(lambda product_title: get_product(product_title, products))

    # Record the end time and calculate the elapsed time
    end_time = time.perf_counter()
    times.append(end_time - start_time)

# Plotting the times against the sample sizes
plt.plot(sample_sizes, times, marker='o')
plt.xlabel('Number of Samples (k)')
plt.ylabel('Time Taken (seconds)')
plt.title('Time vs. Sample Size')
plt.show()

# Does the plot support your thoughts regarding the scale?


### 2. Quick Access using the simulation of an index

Quickmart has grown a lot recently. They have about 2000 product requests every second. They realized that the shop got really slow and customers started to complain. They have the feeling that they could grow even faster but customers are annoyed of the slow shop and therefore prefer to buy electronics somewhere else. What should they do? You remember a very nice module called 'BI and Data Management' during your masters and there you've heard about indices and that those might help to improve the reading times when accessing the database. So let's get to work...

You remember, the first ingredient is the fast access of a database if we use the row number / index. In pandas we can do that by using loc. E.g. to get the 42th entry of the datatable we can call products.loc[42].
It's also possible to access multiple row numbers at once by using an array, e.g. like this: products.loc[[23,42]]

The second ingredient that we need is an auxiliary table which allows us to efficiently 
get the row numbers of the products table given a product title. In the lecture we discussed the binary search.
However, in this exercise we would like to use a dictionary from python. In a python dictionary we have the possible to very efficiently map a given string to a data type of your choice. In our case, we just want to map the product titles to the row numbers.

In [6]:
# Please create the auxiliary table, which can be used to efficiently access the original products table.
products_aux = {}
for index, product_title in products['Product Title'].items():
    products_aux[product_title] = index

# Pythonic short version using dictionary comprehension
products_aux = {product_title: index for index, product_title in products['Product Title'].items()}


In [7]:
# Now that we have the auxiliary table, we design a faster version of the get_products function
# by utilizing this table.

def get_product_fast(product_title, products_aux, products):
    row_index = products_aux[product_title]
    return products.loc[row_index]

In [8]:
# Now it's the moment of truth, was our effort worth it? Let's measure the time for 10000 hits!
k = 10000

# So let's create the random product titles again.
random_product_titles = products['Product Title'].sample(k, replace=True)

In [ ]:
%%time

# Measure the time!
random_product_titles.apply(lambda product_title: get_product_fast(product_title, products_aux, products))

# How long does it take for 1000, 10000, 100000, ...? How does it scale regarding time?

In [ ]:
import time
import matplotlib.pyplot as plt

# Additional Task: Again, automate the process we just did manually.
# Please run it for incremental sample sizes k and create a plot.
# The x axis should be the sample size and the y axis the time it took to get the products.

# Hint: to measure time between lines of code in python, you can use the
# time library. After importing time, time.process_time() returns a timestamp
# in seconds at this very point in time.

# Define the different sample sizes (k values) you want to test
sample_sizes = [1, 10, 100, 1000, 2000, 3000, 4000, 5000, 10000, 20000, 50000, 100000, 150000, 200000]
times = []  # List to store time taken for each sample size

# Loop through each sample size
for k in sample_sizes:
    # Take a sample of 'Product Title' column with size k
    random_product_titles = products['Product Title'].sample(k, replace=True)

    # Record the start time
    start_time = time.perf_counter()

    # Apply the get_product function on each product title in the sample
    random_products = random_product_titles.apply(lambda product_title: get_product_fast(product_title, products_aux, products))

    # Record the end time and calculate the elapsed time
    end_time = time.perf_counter()
    times.append(end_time - start_time)

# Plotting the times against the sample sizes
plt.plot(sample_sizes, times, marker='o')
plt.xlabel('Number of Samples (k)')
plt.ylabel('Time Taken (seconds)')
plt.title('Time vs. Sample Size')
plt.show()

# Does the plot support your thoughts regarding the scale?
# Approximately how many product requests can be processed in one second?


We made it! We solved the issue regarding product access times for Quickmart. You can now make an index proposal with your head held high and put it into production. However, this is beyond our exercise ;)

### Homework

Quickmart has noted that not only with the increased number of customers and requests into the product table everything got slower, but also with the number of products they have in the database. They didn't put a lot of effort to remove discontinued products from the products table. Now they have asked themselves whether it is possible to estimate how much slower the system becomes with an increasing number of products for the slow and fast variants.

To answer this question, we need to fix the number of requests into the data table (.e.g. 10000 for the slow access an 200000 for the quick one) and vary the number of products from the products table. Please try to estimate the influence (the scaling regarding time) when varying the size of the products table (e.g. 1000, 5000, 10000, 15000, 20000, 25000, 30000, 35000).

Additional Task: Provide a plot with the x-axis corresponding to the number of products and the y axis corresponding to the time it took for the request of products. How does time scale with an increased number of products?

## SQL for Beginners

In pandas we can also use SQL to query data. In this exercise, we want to try out a few basic SQL operations and get an idea of its power.